# Demo: Basic Usage

This notebook contains a demonstration of a common usage of CXAS. We display how to segment CXR anatomy in different data types, how to extract features, how to store the predictions, and how to visualize them.

We use a basic frontal CXR as example.

## Installation of CXAS

In [2]:
! pip install cxas

ERROR: Could not find a version that satisfies the requirement cxas (from versions: none)
ERROR: No matching distribution found for cxas


## Import necessary packages

In [3]:
import torch
import numpy as np
from PIL import Image
import shutil
import os

from cxas import CXAS



## Process images

In [ ]:
model = CXAS(
    model_name = 'UNet_ResNet50_default',
    gpus       = '0'
)

# Process tensors

In [3]:
sample_tensor = torch.randn(1,3,512,512).to('cuda:0')

_ = model(sample_tensor)

sample_output = model({'data': sample_tensor})

for k in sample_output.keys():
    if type(sample_output[k]) in [np.ndarray, torch.Tensor]:
        print('{}: \n\t shape - {} \n\t dtype - {} \n'.format(k, sample_output[k].shape, sample_output[k].dtype))
    else:
        print('{}: \n\t example - {}\n\t dtype - {} \n'.format(k, sample_output[k], type(sample_output[k])))


feats: 
	 shape - torch.Size([1, 128, 128, 128]) 
	 dtype - torch.float32 

logits: 
	 shape - torch.Size([1, 159, 512, 512]) 
	 dtype - torch.float32 

data: 
	 shape - torch.Size([1, 3, 512, 512]) 
	 dtype - torch.float32 

segmentation_preds: 
	 shape - torch.Size([1, 159, 512, 512]) 
	 dtype - torch.bool 



# Process Image files

In [4]:
# Dicom images
path = 'images/126_IM-0176-2002.dcm'

sample_output = model.process_file(
        filename = path
        )

#  ---------------------------------------------------------------------

# png  files
path = 'images/00003440_000.png'

sample_output = model.process_file(
        filename = path
        )

#  ---------------------------------------------------------------------

# jpg files
path = 'images/00003440_000.jpg'

sample_output = model.process_file(
        filename = path
        )

#  ---------------------------------------------------------------------


for k in sample_output.keys():
    if type(sample_output[k]) in [np.ndarray, torch.Tensor]:
        print('{}: \n\t shape - {} \n\t dtype - {} \n'.format(k, sample_output[k].shape, sample_output[k].dtype))
    else:
        print('{}: \n\t example - {}\n\t dtype - {} \n'.format(k, sample_output[k], type(sample_output[k])))

GDCMImageIO (0x55e900bdb580): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.



feats: 
	 shape - torch.Size([1, 128, 128, 128]) 
	 dtype - torch.float32 

logits: 
	 shape - torch.Size([1, 159, 512, 512]) 
	 dtype - torch.float32 

data: 
	 shape - torch.Size([1, 3, 512, 512]) 
	 dtype - torch.float32 

orig_data: 
	 shape - (3, 1024, 1024) 
	 dtype - uint8 

filename: 
	 example - ['images/00003440_000.jpg']
	 dtype - <class 'list'> 

file_size: 
	 example - [(1024, 1024)]
	 dtype - <class 'list'> 

segmentation_preds: 
	 shape - torch.Size([1, 159, 512, 512]) 
	 dtype - torch.bool 



# Store Results

In [3]:
path = 'images/126_IM-0176-2002.dcm'
out_path = './out_segs'

print('Store each segmentation label as dicom-seg (.dcm) in folder \n \
        This is only possible for dicom files!')
print('Does the out_path exists:', os.path.isdir(out_path))

_ = model.process_file(
        filename = path,
        do_store = True, 
        output_directory = out_path,
        storage_type = 'dicom-seg',
        )

print('First five Files in directory:', [os.path.join(dp, f) 
                              for dp, dn, fn in os.walk(os.path.expanduser(out_path)) for f in fn][:5])
shutil.rmtree(out_path)

print('Store as .npy-file')
print('Does the out_path exists:', os.path.isdir(out_path))

_ = model.process_file(
        filename = path,
        do_store = True, 
        output_directory = out_path,
        storage_type = 'npy',
        )

print('Files in directory:', os.listdir(out_path))
shutil.rmtree(out_path)

print('Store as .npz-file')
print('Does the out_path exists:', os.path.isdir(out_path))

_ = model.process_file(
        filename = path,
        do_store = True, 
        output_directory = out_path,
        storage_type = 'npz',
        )

print('Files in directory:', os.listdir(out_path))
shutil.rmtree(out_path)

print('Store each segmentation label as jpg in folder')
print('Does the out_path exists:', os.path.isdir(out_path))

_ = model.process_file(
        filename = path,
        do_store = True, 
        output_directory = out_path,
        storage_type = 'jpg',
        )

print('First five Files in directory:', [os.path.join(dp, f) 
                              for dp, dn, fn in os.walk(os.path.expanduser(out_path)) for f in fn][:5])
shutil.rmtree(out_path)

print('Store segmentation as json in coco format')
print('Does the out_path exists:', os.path.isdir(out_path))
_ = model.process_file(
        filename = path,
        do_store = True, 
        output_directory = out_path,
        storage_type = 'json',
        )

print('Files in directory:', os.listdir(out_path))

shutil.rmtree(out_path)



Store each segmentation label as dicom-seg (.dcm) in folder 
         This is only possible for dicom files!
Does the out_path exists: True


GDCMImageIO (0x55bd50fbfa40): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55bdcc4bb1f0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.



First five Files in directory: ['./out_segs/126_IM-0176-2002/i_anterior 1st rib left.dcm', './out_segs/126_IM-0176-2002/i_anterior 1st rib right.dcm', './out_segs/126_IM-0176-2002/i_esophagus.dcm', './out_segs/126_IM-0176-2002/i_posterior 4th rib.dcm', './out_segs/126_IM-0176-2002/i_small bowel.dcm']
Store as .npy-file
Does the out_path exists: False


GDCMImageIO (0x55bde8ed7f90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.



Files in directory: ['126_IM-0176-2002.npy']
Store as .npz-file
Does the out_path exists: False


...

Files in directory: ['126_IM-0176-2002.npz']
Store each segmentation label as jpg in folder
Does the out_path exists: False


GDCMImageIO (0x55bde8efd590): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.



First five Files in directory: ['./out_segs/126_IM-0176-2002/126_IM-0176-2002_anterior 8th rib left.jpg', './out_segs/126_IM-0176-2002/126_IM-0176-2002_lung base.jpg', './out_segs/126_IM-0176-2002/126_IM-0176-2002_vertebrae T9.jpg', './out_segs/126_IM-0176-2002/126_IM-0176-2002_pulmonary artery.jpg', './out_segs/126_IM-0176-2002/126_IM-0176-2002_vertebrae C3.jpg']
Store segmentation as json in coco format
Does the out_path exists: False


GDCMImageIO (0x55bdcd00d5d0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.



Files in directory: ['126_IM-0176-2002.json']
